In [19]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data

def val_missing_data_rbf(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    imputations, x_loss, c_hat_list = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [20]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_1000_0.05_time.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 64)

In [21]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = "cuda" if torch.cuda.is_available() else "cpu"

class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

# Generator 모델
class MGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            hid = hid * gamma_h

            r_hat = self.temp_decay_r(r)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            RG = torch.cat([x_hat, r_hat], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * x_hat

            z_hat = self.z_layer(x_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return imputations, x_loss, c_hat_list
    
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.grucell = nn.GRUCell(input_size, hidden_size)
        
        self.fc = nn.Linear(hidden_size, input_size)

    
    def forward(self, X):
        hid = torch.zeros((X.size(0), self.hidden_size)).cuda()
        D_list = []

        for i in range(X.size(1)):

            gru_input = X[:,i,:]
            Y_hat = torch.sigmoid(self.fc(hid))

            hid = self.grucell(gru_input, hid)
            D_list.append(Y_hat.unsqueeze(1))
        
        D_list = torch.cat(D_list, dim = 1)

        return D_list

In [22]:
def train_MGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        for data in dataset:
            data = data.to(device)
            imputations, x_loss, c_hat_list = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
        progress.set_description("loss: {}".format(batch_loss))

    return x_loss
    

In [23]:
def train_MGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        for data in dataset:
            data = data.to(device)
            imputations, x_loss, c_hat_list = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
        progress.set_description("loss: {}".format(batch_loss))

    return x_loss
    

In [25]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 102.03832244873047: 100%|██████████| 1000/1000 [14:28<00:00,  1.15it/s]


Scale MAE : tensor(0.1931)
Scale MRE : tensor(0.2788)
Original MAE : 15.602317905134681
Original MRE : 0.21916334266558632
Train MAE : 9.867832214826137
Train MRE : 0.1164007089212494


In [7]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 97.75187683105469: 100%|██████████| 1000/1000 [14:21<00:00,  1.16it/s]


Scale MAE : tensor(0.1768)
Scale MRE : tensor(0.2553)
Original MAE : 14.2635652756038
Original MRE : 0.20035809186411582


In [8]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 108.84825134277344: 100%|██████████| 1000/1000 [14:32<00:00,  1.15it/s]


Scale MAE : tensor(0.1717)
Scale MRE : tensor(0.2479)
Original MAE : 13.85137359181268
Original MRE : 0.19456809913432477


In [9]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 108.54731750488281: 100%|██████████| 1000/1000 [14:26<00:00,  1.15it/s]


Scale MAE : tensor(0.1736)
Scale MRE : tensor(0.2506)
Original MAE : 13.97785646138444
Original MRE : 0.19634478440979655


In [17]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 113.53192138671875: 100%|██████████| 1000/1000 [12:54<00:00,  1.29it/s]


Scale MAE : tensor(0.1753)
Scale MRE : tensor(0.2531)
Original MAE : 14.139500094776402
Original MRE : 0.19861536748791309
Train MAE : 9.671704676288563
Train MRE : 0.11408719324447768


In [18]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 97.75187683105469: 100%|██████████| 1000/1000 [14:37<00:00,  1.14it/s]


Scale MAE : tensor(0.1768)
Scale MRE : tensor(0.2553)
Original MAE : 14.2635652756038
Original MRE : 0.20035809186411582
Train MAE : 9.504182306608
Train MRE : 0.11211110344415785


In [26]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 64)

In [27]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 102.06137084960938: 100%|██████████| 1000/1000 [12:51<00:00,  1.30it/s]


Scale MAE : tensor(0.1824)
Scale MRE : tensor(0.2634)
Original MAE : 14.786748656331092
Original MRE : 0.2077071677670969
Train MAE : 9.859373299496665
Train MRE : 0.11630092776165732
